In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from astropy.cosmology import LambdaCDM

from scipy.optimize import curve_fit

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)

In [2]:
# calling necessary class from ler package
from ler.gw_source_population import CBCSourceRedshiftDistribution

# uncomment the following line to see the docstring
# SourceGalaxyPopulationModel?

# class initialization
# default model "BBH popI/II Oguri2018"
cbc = CBCSourceRedshiftDistribution()

# list out the models for the merger rate density wrt redshift
print("\n available model list with it input parameters: \n", cbc.merger_rate_density_model_list)

z_to_luminosity_distance interpolator will be loaded from ./interpolator_pickle/z_to_luminosity_distance/z_to_luminosity_distance_3.pickle
differential_comoving_volume interpolator will be loaded from ./interpolator_pickle/differential_comoving_volume/differential_comoving_volume_1.pickle
merger_rate_density interpolator will be generated at ./interpolator_pickle/merger_rate_density/merger_rate_density_2.pickle

 available model list with it input parameters: 
 {'merger_rate_density_bbh_popI_II_oguri2018': {'R0': 2.39e-08, 'b2': 1.6, 'b3': 2.0, 'b4': 30}, 'star_formation_rate_madau_dickinson2014': {'af': 2.7, 'bf': 5.6, 'cf': 2.9}, 'merger_rate_density_bbh_popIII_ken2022': {'n0': 1.92e-08, 'aIII': 0.66, 'bIII': 0.3, 'zIII': 11.6}, 'merger_rate_density_bbh_primordial_ken2022': {'n0': 4.4e-11, 't0': 13.786885302009708}}


In [4]:
Model_name = ['M13A', 'M23A', 'M25B', 'M30B', 'M33A', 'M40B', 'M43A', 'M50B', 'M60B', 'M70B']
R0_BNS = [193.0, 204.0, 105.0, 122.0, 321.0, 122.0, 323.0, 93.2, 118.0, 120.0]  ##in Gpc^-3yr^-1

param_array = np.zeros(3)

for i in  range(len(Model_name)):
	print('%s: %s'%(Model_name[i], R0_BNS[i]))

	R0 = (R0_BNS[i]) # ##in Mpc^-3yr^-1


	#dVc_dz = cbc.differential_comoving_volume(z)
	def merger_density_fit(x, b2, b3, b4):
		

		return  (R0 * (b4 + 1) * np.exp(b2 * x) / (b4 + np.exp(b3 * x))) 



	xdata = np.linspace(0., 2.0, 1000)
	cbc = CBCSourceRedshiftDistribution()
	dvdz = (cbc.differential_comoving_volume(xdata))*((10**-3)**3)  ##dvdz is originally in Mpc^3;convert to Gpc^3
	ydata = dvdz * (R0*((1+xdata)**1.5)) / (1+xdata)   ##convert density to detector frame
	plt.plot(xdata, ydata/(10**9), 'b-', label='data')  #plot in Mpc^-3 yr ^-1

	popt, pcov = curve_fit(merger_density_fit, xdata, ydata, method ='dogbox')

	print(popt)
	param_array = np.vstack((param_array, popt))
	plt.plot(xdata, merger_density_fit(xdata, *popt)/(10**9), 'r-', label='fit: b2=%5.3f, b3=%5.3f, b4=%5.3f' % tuple(popt))  #plot in Mpc^-3 yr ^-1

	#Constrain the optimization to the region of 0 <= a <= 3, 0 <= b <= 1 and 0 <= c <= 0.5:

	#popt, pcov = curve_fit(merger_density_fit, xdata, ydata, bounds=([0, -2, 0], [1., 1, 0.005]),method ='dogbox')

	#print(popt)


	#plt.plot(xdata, merger_density_fit(xdata, *popt), 'g--', label='fit: b2=%5.3f, b3=%5.3f, b4=%5.3f' % tuple(popt))

	plt.xlabel('redshift')

	plt.ylabel(r'merger rate density $\rm [Mpc^{-3} yr^{-1}]$')
	plt.yscale('log')
	plt.legend()
	plt.savefig('merger_rate_bns_%s'%(Model_name[i]))
	#plt.show()
	plt.close()


param_array = param_array[1:]
print(param_array.shape)


data = np.load('/home/neha/Desktop/Chris_BNS_models/m13A/m13_data.npy')       #np.array([i for i in zip(Ma, Mb, zmer, ri, h_indx)]).T

Ma, Mb, zmer, ri, h_indx = data

c1 = Ma<2.5
c2 = Mb<2.5


bns_mass_model = np.array([i for i in zip(Ma[c1*c2], Mb[c1*c2])])
print(bns_mass_model.shape)
np.save('./masses_m13',bns_mass_model)

sorted_mass = np.ones(bns_mass_model.shape)

sorted_mass =  np.array([ [i.max(), i.min()] for i in bns_mass_model])



from ler.rates import LeR
from ler.utils import get_param_from_json

np.random.seed(250)

def masses_model(size, param=None):
    indx = np.random.randint(0, len(sorted_mass), size)
    
    mass1, mass2 = sorted_mass[indx][:,0], sorted_mass[indx][:,1]

    return mass1, mass2


##ler = LeR(verbose=False, event_type='BNS', mtot_max=51, z_max=10)

print(xxx)

m_tot_max = ((bns_mass_model[:,0] + bns_mass_model[:,1]).max()) *(1+2.0)

M13A: 193.0
z_to_luminosity_distance interpolator will be loaded from ./interpolator_pickle/z_to_luminosity_distance/z_to_luminosity_distance_3.pickle
differential_comoving_volume interpolator will be loaded from ./interpolator_pickle/differential_comoving_volume/differential_comoving_volume_1.pickle


AttributeError: Can't get attribute 'star_formation_rate_madau_dickinson2014' on <module '__main__'>

In [ ]:
ler = LeR(npool=8, verbose=True, event_type = 'BNS', mtot_max=m_tot_max, z_max = 2.0,
    # for source parameters
    
    
    source_priors=dict(
        merger_rate_density='merger_rate_density_bbh_popI_II_oguri2018',
        source_frame_masses=masses_model,
    ),
    
    
    source_priors_params = dict( 
       merger_rate_density = dict(R0=(R0_BNS[0])*1e-9, b2=param_array[0][0], b3=param_array[0][1], b4=param_array[0][2] )
    ),
       
    # for lens parameters
    lens_functions=dict(
        strong_lensing_condition="rjs_with_cross_section_SIS",
        optical_depth="optical_depth_SIS_haris",
    ),
    lens_priors=dict(
        velocity_dispersion="velocity_dispersion_gengamma",
    ),
    lens_priors_params=dict(
        velocity_dispersion=dict(a=2.32 / 2.67, c=2.67)
    ),
    
    # for snr generation
    waveform_approximant = 'IMRPhenomXPHM',
    snr_type='inner_product',
    spin_zero=False,
    spin_precession=True,
    psds = dict( L1 = 'aligo_O4low_asd.txt', H1 =  'aligo_O4low_asd.txt'  ),
    ifos = ['L1']#, 'H1'],
)

# ler.batch_size = 100000 # for faster computation
#unlensed_param = ler.unlensed_cbc_statistics(size=5000, resume=False, save_batch=False)


#lensed_param = ler.lensed_cbc_statistics(size=5000, resume=False, save_batch=False)

#=====================================================================================#
#=====================================================================================#
# snr_cut=8.0
ler.selecting_n_unlensed_detectable_events(size=100, batch_size=1000, snr_threshold=4.0, output_jsonfile='n_unlensed_detectable_bns.json', meta_data_file='n_unlensed_detectable_bns_meta.json', resume=True);


ler.selecting_n_lensed_detectable_events(size=100, batch_size=2000, snr_threshold=8.0, num_img=2, output_jsonfile='n_lensed_detectable_bns.json', meta_data_file='n_lensed_detectable_bns_meta.json', resume=True);